In [1]:
import geopandas as gpd
import numpy as np
import json
from shapely.geometry import Polygon, MultiPolygon, LineString, Point, MultiLineString, MultiPoint
from shapely.geometry.base import BaseGeometry

def shift_longitude(geom, center_lon=105):
    """
    调整几何图形的经度，使特定经度位于中心

    参数:
    geom - shapely几何对象
    center_lon - 目标中心经度（默认为中国中心，约105度）

    返回:
    调整后的几何对象
    """
    if geom is None:
        return None

    # 计算需要移动的经度量
    shift = center_lon

    # 创建一个函数来处理单个坐标
    def shift_point(x, y):
        # 将经度调整为以指定经度为中心
        new_x = ((x + 180 - shift) % 360) - 180 + shift
        return new_x, y

    # 处理不同类型的几何图形
    if isinstance(geom, Point):
        x, y = geom.x, geom.y
        new_x, new_y = shift_point(x, y)
        return Point(new_x, new_y)

    elif isinstance(geom, LineString):
        new_coords = [shift_point(x, y) for x, y in geom.coords]
        return LineString(new_coords)

    elif isinstance(geom, Polygon):
        exterior = [(shift_point(x, y)) for x, y in geom.exterior.coords]
        interiors = [[(shift_point(x, y)) for x, y in interior.coords]
                     for interior in geom.interiors]
        return Polygon(exterior, interiors)

    elif isinstance(geom, MultiPoint):
        points = [Point(shift_point(p.x, p.y)) for p in geom.geoms]
        return MultiPoint(points)

    elif isinstance(geom, MultiLineString):
        lines = [LineString([shift_point(x, y) for x, y in line.coords])
                for line in geom.geoms]
        return MultiLineString(lines)

    elif isinstance(geom, MultiPolygon):
        polygons = []
        for poly in geom.geoms:
            exterior = [(shift_point(x, y)) for x, y in poly.exterior.coords]
            interiors = [[(shift_point(x, y)) for x, y in interior.coords]
                         for interior in poly.interiors]
            polygons.append(Polygon(exterior, interiors))
        return MultiPolygon(polygons)

    else:
        raise ValueError(f"Unsupported geometry type: {type(geom)}")

def process_geojson(input_file, output_file, center_lon=105):
    """
    处理GeoJSON文件，使地图以指定经度为中心

    参数:
    input_file - 输入的GeoJSON文件路径
    output_file - 输出的GeoJSON文件路径
    center_lon - 目标中心经度（默认为中国中心，约105度）
    """
    # 读取GeoJSON文件
    world = gpd.read_file(input_file)

    # 应用坐标转换
    world['geometry'] = world['geometry'].apply(lambda geom: shift_longitude(geom, center_lon))

    # 处理可能的日期线分割问题
    # 当地图被调整中心时，一些跨越边界的多边形可能需要特殊处理

    # 保存为新的GeoJSON文件
    world.to_file(output_file, driver='GeoJSON')
    print(f"已创建以中国为中心的世界地图: {output_file}")

def download_world_map():
    """
    从Natural Earth下载世界地图数据

    返回:
    临时文件路径
    """
    import tempfile
    import os
    from urllib.request import urlretrieve

    # 创建临时文件
    temp_dir = tempfile.gettempdir()
    input_file = os.path.join(temp_dir, "ne_110m_admin_0_countries.geojson")

    # 如果文件不存在，则下载
    if not os.path.exists(input_file):
        url = "https://raw.githubusercontent.com/nvkelso/natural-earth-vector/master/geojson/ne_110m_admin_0_countries.geojson"
        print(f"下载世界地图数据...")
        urlretrieve(url, input_file)

    return input_file

def main():
    """
    主函数 - 下载世界地图并处理为以中国为中心
    """
    try:
        # 下载或使用现有的世界地图数据
        input_file = download_world_map()
        output_file = "world_map_china_centered.geojson"

        # 处理为以中国为中心的地图
        process_geojson(input_file, output_file, center_lon=105)

    except Exception as e:
        print(f"处理地图数据时出错: {e}")

if __name__ == "__main__":
    main()

下载世界地图数据...
已创建以中国为中心的世界地图: world_map_china_centered.geojson


In [4]:
import json
import geopandas as gpd
from shapely.geometry import shape, mapping
from shapely.validation import explain_validity, make_valid

def fix_invalid_geometries(input_file, output_file):
    """
    修复GeoJSON文件中的无效几何形状

    参数:
    input_file - 输入的GeoJSON文件路径
    output_file - 输出的GeoJSON文件路径
    """
    print(f"正在处理 {input_file}...")

    # 读取GeoJSON文件
    try:
        # 使用geopandas读取文件
        gdf = gpd.read_file(input_file)
        print(f"成功读取文件，包含 {len(gdf)} 个地理要素")
    except Exception as e:
        # 如果geopandas读取失败，尝试手动读取并修复
        print(f"geopandas读取失败，尝试手动修复: {e}")
        with open(input_file, 'r', encoding='utf-8') as f:
            geojson_data = json.load(f)

        # 修复每个要素的几何形状
        fixed_features = []
        for feature in geojson_data['features']:
            try:
                geom = shape(feature['geometry'])

                # 检查几何形状是否有效
                if not geom.is_valid:
                    # 输出无效原因
                    reason = explain_validity(geom)
                    print(f"发现无效几何形状: {reason}")

                    # 尝试修复
                    fixed_geom = make_valid(geom)
                    feature['geometry'] = mapping(fixed_geom)
                    print("已修复几何形状")

                fixed_features.append(feature)
            except Exception as e:
                print(f"修复几何形状时出错: {e}")
                # 如果无法修复，可以选择跳过该要素或保留原始要素
                fixed_features.append(feature)

        # 创建新的GeoJSON
        geojson_data['features'] = fixed_features

        # 保存修复后的文件
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(geojson_data, f)

        print(f"已保存修复后的GeoJSON到 {output_file}")
        return

    # 检查并修复无效几何形状
    invalid_count = 0
    for idx, row in gdf.iterrows():
        geom = row.geometry
        if not geom.is_valid:
            invalid_count += 1
            reason = explain_validity(geom)
            print(f"要素 {idx}: {reason}")

            # 修复无效几何形状
            gdf.loc[idx, 'geometry'] = make_valid(geom)

    print(f"发现并修复了 {invalid_count} 个无效几何形状")

    # 保存修复后的GeoJSON
    gdf.to_file(output_file, driver="GeoJSON")
    print(f"已保存修复后的GeoJSON到 {output_file}")

def fix_dateline_crossing_polygons(input_file, output_file, center_lon=105):
    """
    修复跨日期线的多边形问题，并调整为以指定经度为中心

    参数:
    input_file - 输入的GeoJSON文件路径
    output_file - 输出的GeoJSON文件路径
    center_lon - 目标中心经度，默认为105（中国中心）
    """
    # 读取GeoJSON文件
    with open(input_file, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # 处理每个要素
    for feature in data['features']:
        if 'geometry' in feature and feature['geometry']:
            geom_type = feature['geometry']['type']

            # 处理多边形类型的几何形状
            if geom_type == 'Polygon':
                for ring in feature['geometry']['coordinates']:
                    fix_ring_coordinates(ring, center_lon)

            # 处理多多边形类型的几何形状
            elif geom_type == 'MultiPolygon':
                for polygon in feature['geometry']['coordinates']:
                    for ring in polygon:
                        fix_ring_coordinates(ring, center_lon)

    # 保存修复后的文件
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=2)

    print(f"已处理跨日期线问题并保存到 {output_file}")

def fix_ring_coordinates(ring, center_lon=105):
    """
    修复环坐标，处理跨日期线问题并调整中心

    参数:
    ring - 坐标环（列表形式）
    center_lon - 目标中心经度
    """
    # 确保环是闭合的（首尾点一致）
    if ring[0] != ring[-1]:
        ring.append(ring[0].copy())

    # 检测是否有大跨度（可能是跨越日期线）
    prev_x = None
    for i, point in enumerate(ring):
        x, y = point

        # 将经度调整为以指定经度为中心
        shifted_x = ((x + 180 - center_lon) % 360) - 180 + center_lon

        # 检测大跨度并修复
        if prev_x is not None and abs(shifted_x - prev_x) > 180:
            # 需要修复跨越日期线的问题
            if shifted_x < prev_x:
                shifted_x += 360
            else:
                shifted_x -= 360

        # 更新点坐标
        point[0] = shifted_x
        prev_x = shifted_x

def main():
    try:
        # 修复无效几何形状
        fix_invalid_geometries("world_map_china_centered.geojson", "cn22.geojson")

        # 修复跨日期线问题并调整中心为中国
        fix_dateline_crossing_polygons("cn22.geojson", "cn33.geojson", center_lon=105)

        print("已完成所有修复和调整！")
    except Exception as e:
        print(f"处理过程中出错: {e}")

if __name__ == "__main__":
    main()

正在处理 world_map_china_centered.geojson...
成功读取文件，包含 177 个地理要素
要素 0: Self-intersection[180 -16.555217]
要素 3: Self-intersection[279.649952370493 83.0662342943312]
要素 4: Self-intersection[236.223766456996 39.2414224020953]
要素 10: Self-intersection[-71.9539507291146 -46.9439298254235]
要素 14: Self-intersection[33.9633927979515 9.46428502886449]
要素 18: Self-intersection[180 70.832199]
要素 31: Self-intersection[-69.0388410604864 -16.3459115079082]
要素 32: Self-intersection[-72.2668137240885 11.0796363948411]
要素 47: Self-intersection[281.912229942958 20.7348139476293]
要素 159: Ring Self-intersection[180 -84.71338]
发现并修复了 10 个无效几何形状
已保存修复后的GeoJSON到 cn22.geojson
已处理跨日期线问题并保存到 cn33.geojson
已完成所有修复和调整！
